## Imports

In [10]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [39]:
import json
import os

import torch
from datasets import Dataset
from safetensors.torch import load_file

from bergson.collection import collect_gradients
from bergson.data import DataConfig, IndexConfig, load_gradients
from bergson.distributed import distributed_computing, setup_data_pipeline

## -1. Compute gradients

In [28]:
test_ekfac_path = "/root/bergson-approx-unrolling/tests/ekfac_tests/test_files/pile_100_examples"
test_gradients_path = test_ekfac_path + "/test_gradients"

In [30]:
cfg_json = json.load(open(os.path.join(test_ekfac_path, "ground_truth", "index_config.json"), "r"))
cfg = IndexConfig(**cfg_json)
cfg.data = DataConfig(**(cfg_json["data"]))


In [41]:
data = setup_data_pipeline(cfg)
assert isinstance(data, Dataset)
data = data.select(range(10))  # only a small number of examples since we also want to store uncompressed gradients
# save data to test_gradients_path
test_gradients_path = os.path.join(test_gradients_path, "gradient_data")
data.save_to_disk(test_gradients_path)

Saving the dataset (0/1 shards):   0%|          | 0/10 [00:00<?, ? examples/s]

In [48]:
cfg.ekfac = True
cfg.skip_preconditioners = True
cfg.world_size = 1
cfg.data.dataset = test_gradients_path
cfg.data.completion_column = ""
cfg.data.conversation_column = ""


In [50]:
test_gradients_path

'/root/bergson-approx-unrolling/tests/ekfac_tests/test_files/pile_100_examples/gradients/gradient_data'

In [38]:
# Run compressed version
cfg.run_path = test_gradients_path
cfg.projection_dim = 16

In [51]:
setup_data_pipeline(cfg)

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

KeyError: ''

In [49]:
distributed_computing(
    cfg=cfg,
    worker_fn=collect_gradients,
)

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

KeyError: ''

## 0. Load EKFAC

In [ ]:
ekfac_path = ""

# all paths inside ekfac_path
world_size = len(os.listdir(ekfac_path + "/activation_covariance_sharded"))

eigen_a_paths = [ekfac_path + f"/activation_eigen_sharded/shard_{rank}.safetensors" for rank in range(world_size)]
eigen_a = [load_file(path, device="cuda") for path in eigen_a_paths]

eigen_g_paths = [ekfac_path + f"/gradient_eigen_sharded/shard_{rank}.safetensors" for rank in range(world_size)]
eigen_g = [load_file(path, device="cuda") for path in eigen_g_paths]

lambda_factor_paths = [
    ekfac_path + f"/eigenvalue_correction_sharded/shard_{rank}.safetensors" for rank in range(world_size)
]
lambda_factor = [load_file(path, device="cuda") for path in lambda_factor_paths]


In [15]:
ekfac_path = "/root/bergson-approx-unrolling/bergson/hessians/peft_fin_mis_fin/influence_results"
world_size = len(os.listdir(ekfac_path + "/activation_covariance_sharded"))
lambda_factor_paths = [
    ekfac_path + f"/inverse_eigenvalue_correction_sharded/shard_{rank}.safetensors" for rank in range(world_size)
]
lambda_factor = [load_file(path, device="cuda") for path in lambda_factor_paths]
lambda_factor_tensor = {}
for k, v in lambda_factor[0].items():
    lambda_factor_tensor[k] = torch.cat([lambda_factor[rank][k] for rank in range(world_size)], dim=0)


In [16]:
for k, v in lambda_factor_tensor.items():
    print(k, v.mean().item(), v.std().item(), v.min().item(), v.max().item())

layers.0.mlp.down_proj.lora_A.default 10.0 0.0 10.0 10.0
layers.0.mlp.down_proj.lora_B.default 10.0 0.0 10.0 10.0
layers.0.mlp.gate_proj.lora_A.default 10.0 0.0 10.0 10.0
layers.0.mlp.gate_proj.lora_B.default 10.0 0.0 10.0 10.0
layers.0.mlp.up_proj.lora_A.default 10.0 0.0 10.0 10.0
layers.0.mlp.up_proj.lora_B.default 10.0 0.0 10.0 10.0
layers.1.mlp.down_proj.lora_A.default 10.0 0.0 10.0 10.0
layers.1.mlp.down_proj.lora_B.default 10.0 0.0 10.0 10.0
layers.1.mlp.gate_proj.lora_A.default 10.0 0.0 10.0 10.0
layers.1.mlp.gate_proj.lora_B.default 10.0 0.0 10.0 10.0
layers.1.mlp.up_proj.lora_A.default 10.0 0.0 10.0 10.0
layers.1.mlp.up_proj.lora_B.default 10.0 0.0 10.0 10.0
layers.10.mlp.down_proj.lora_A.default 10.0 0.0 10.0 10.0
layers.10.mlp.down_proj.lora_B.default 10.0 0.0 10.0 10.0
layers.10.mlp.gate_proj.lora_A.default 10.0 0.0 10.0 10.0
layers.10.mlp.gate_proj.lora_B.default 10.0 0.0 10.0 10.0
layers.10.mlp.up_proj.lora_A.default 10.0 0.0 10.0 10.0
layers.10.mlp.up_proj.lora_B.default

## 1. Load the gradient


In [ ]:
gradient_path = ""

mmap = load_gradients(gradient_path)
with open(os.path.join(gradient_path, "info.json")) as f:
    info = json.load(f)


NameError: name 'load_gradients' is not defined

## 2. Apply EKFAC